In [1]:
%matplotlib inline
import gc
import os
import glob
import fitsio
import numpy as np
#import dask.dataframe as ddf
import matplotlib.pyplot as plt

In [2]:
fields=['XMM','VVDS','HECTOMAP','GAMA09H','WIDE12H','GAMA15H']
wrkDir='/hildafs/projects/phy200017p/share/'
outDir=os.path.join(wrkDir,'raytracesim/HSC_S16A/mocks/')

In [3]:
mockDir=os.path.join(wrkDir,'raytracesim/HSC_S16A')
for isim in range(25): #108 n-body simulations
    for irot in range(21): # 21 rotations
        outfname=os.path.join(outDir,'r%03d_rot%d.parquet' %(isim,irot))
        if os.path.isdir(outfname):
            continue
        datFin=[]
        first =True # first in fields
        for fieldname in fields:
            datAll=[]
            sDir= os.path.join(mockDir,fieldname,'r%03d'%isim, 'rotmat%d' %irot,'*.fits')
            fnamelist=glob.glob(sDir)
            for fname in fnamelist:
                data=fitsio.read(fname)
                datAll.append(data)
                del data
                os.remove(fname)
            del fnamelist
            tmp=ddf.from_array(np.hstack(datAll).byteswap().newbyteorder())
            del datAll
            tmp['field']=fieldname
            if first:
                datFin=tmp
                first=False
            else:
                datFin=ddf.concat([datFin,tmp])
            del tmp
            gc.collect()
         
        datFin['field'].astype("string[pyarrow]")
        datFin.to_parquet(outfname,overwrite=True)
        del datFin
        gc.collect()

In [4]:
mockDir=os.path.join(wrkDir,'raytracesim/HSC_S16A')
for isim in range(108): #108 n-body simulations
    for irot in range(21): # 21 rotations
        outfname=os.path.join(outDir,'r%03d_rot%d.parquet' %(isim,irot))
        if os.path.isdir(outfname):
            for fieldname in fields:
                sDir= os.path.join(mockDir,fieldname,'r%03d'%isim, 'rotmat%d' %irot,'*.fits')
                fnamelist=glob.glob(sDir)
                for fname in fnamelist:
                    if os.path.isfile(fname):
                        os.remove(fname)

In [5]:
obsDir='/hildafs/projects/phy200017p/share/HSC_shape_catalog/'
outDir='/hildafs/projects/phy200017p/share/raytracesim/HSC_S16A/s16a_data/'
outfname=os.path.join(outDir,'shape_calibrated.parquet')

datFin=[]
for fieldname in fields:
    datAll=[]
    fname= os.path.join(obsDir,'%s_calibrated.fits' %fieldname)
    tmp=ddf.from_array(fitsio.read(fname).byteswap().newbyteorder())
    tmp['field']=fieldname
    datFin.append(tmp)
    del tmp
gc.collect()
datFin=ddf.concat(datFin)
datFin['field'].astype("string[pyarrow]")
datFin.set_index('object_id')
datFin.to_parquet(outfname,overwrite=True)
del datFin
gc.collect()

0

In [6]:
# photoz
obsDir='/hildafs/projects/phy200017p/share/HSC_shape_catalog/'
outDir='/hildafs/projects/phy200017p/share/raytracesim/HSC_S16A/s16a_data/'
outfname=os.path.join(outDir,'photoz.parquet')

datFin=[]
for fieldname in fields:
    datAll=[]
    fname= os.path.join(obsDir,'%s_calibrated_pz.fits' %fieldname)
    tmp=ddf.from_array(fitsio.read(fname).byteswap().newbyteorder())
    tmp['field']=fieldname
    datFin.append(tmp)
    del tmp
gc.collect()
datFin=ddf.concat(datFin)
datFin['field'].astype("string[pyarrow]")
datFin.set_index('object_id').to_parquet(outfname,overwrite=True)
del datFin
gc.collect()

0

In [ ]:
for isim in range(25): #108 n-body simulations
    for irot in range(21): # 21 rotations
        outfname=os.path.join(outDir,'r%03d_rot%d.parquet' %(isim,irot))
        assert os.path.isdir(outfname)
        mock=ddf.read_parquet(outfname)
        mock.set_index('object_id').to_parquet(outfname)
        del mock
    gc.collect()